In [1]:
# Import necessary libraries
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import ipywidgets as widgets
from ipywidgets import Layout, interactive_output, FloatSlider, HBox, VBox, Dropdown, Button, Label
from IPython.display import display, clear_output

# Set plot settings
plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'
plt.rcParams['figure.figsize'] = [10, 4.5]

# Initial triangle coordinates
triangle = np.array([[0, 0], [1, 0], [0.5, 1]])

def plot_shape(ax, shape, *args, **kwargs):
    """Function to plot a shape."""
    shape_closed = np.concatenate((shape, shape[:1]), axis=0)
    ax.plot(shape_closed[:, 0], shape_closed[:, 1], *args, **kwargs)

def plot_matrix(ax, mat, color_array):
    """Function to plot a matrix."""
    ax.axis('tight')
    ax.axis('off')
    formatted_mat = [[f'{value:.2f}' for value in row] for row in mat]  
    tb = ax.table(cellText=formatted_mat, cellColours=color_array,
                  loc='center', cellLoc='center', rowLabels=[''] * 3, bbox=[0.1, 0.6, 0.7, 0.4])
    tb.auto_set_font_size(False)
    tb.set_fontsize(20)
    tb.scale(1.5, 2)

def update(shape_type='Triangle', scale=1.0, rotation_angle=0.0, trans_x=0.0, trans_y=0.0):
    """Update function for interactive plot."""
    fig = plt.figure(figsize=(14, 6))
    gs = gridspec.GridSpec(2, 2, height_ratios=[1, 2], width_ratios=[3, 1])
    
    ax = plt.subplot(gs[:, 0])
    ax_matrix = plt.subplot(gs[1, 1])
    ax_latex = plt.subplot(gs[0, 1])
    
    ax_latex.axis('off')
    
    # Set axis properties
    ax.grid(True)
    ax.set_aspect('equal')
    ax.set_xlim([-2, 2])
    ax.set_ylim([-2, 2])

    # Create transformation matrix
    rotation_matrix = np.array([
        [np.cos(np.radians(rotation_angle)), -np.sin(np.radians(rotation_angle)), trans_x],
        [np.sin(np.radians(rotation_angle)), np.cos(np.radians(rotation_angle)), trans_y],
        [0, 0, 1]
    ])
    scaling_matrix = np.array([[scale, 0, 0], [0, scale, 0], [0, 0, 1]])
    transformation_matrix = np.dot(rotation_matrix, scaling_matrix)

    # Determine which cells to color based on the last slider moved
    color_array = np.full((3, 3, 3), 1.0)  # default color
    color_array[2, 0] = color_array[2, 1] = color_array[2, 2] = [0.8, 0.8, 0.8] 
    if last_moved_slider == "Angle":
        color_array[0:2, 0:2] = [0.8, 1, 0.8]  # green for rotation
    elif last_moved_slider == "Translation X":
        color_array[0, 2] = [0.8, 1, 0.8]  # green for translation in X
    elif last_moved_slider == "Translation Y":
        color_array[1, 2] = [0.8, 1, 0.8]  # green for translation in Y
    elif last_moved_slider == "Scale":
        color_array[0, 0] = color_array[1, 1] = [0.8, 1, 0.8]  # green for scaling


    # Determine which shape to plot based on the selection
    if shape_type == 'Triangle':
        # Plot original triangle
        plot_shape(ax, triangle, 'k--')  # Plotting in black dashed line
        # Convert triangle to homogeneous coordinates
        homog_triangle = np.concatenate((triangle, np.ones((3, 1))), axis=1)
        # Apply transformation matrix
        transformed_triangle_homog = np.dot(homog_triangle, transformation_matrix.T)
        # Convert back to Cartesian coordinates
        transformed_triangle = transformed_triangle_homog[:, :2]
        # Plot transformed shape
        plot_shape(ax, transformed_triangle, 'r-')
    elif shape_type == 'Coordinate System':

        # Define original coordinate system
        origin = np.array([[0, 0, 1]])  # origin point in homogeneous coordinates
        x_axis = np.array([[1, 0, 1]])  # x-axis endpoint in homogeneous coordinates
        y_axis = np.array([[0, 1, 1]])  # y-axis endpoint in homogeneous coordinates

        # Plot original coordinate system using quiver
        ax.quiver(origin[0, 0], origin[0, 1], x_axis[0, 0]-origin[0, 0], x_axis[0, 1]-origin[0, 1],
                  angles='xy', scale_units='xy', scale=1, color=(1, 0, 0, 0.3), width=0.02)  # x-axis in blue
        ax.quiver(origin[0, 0], origin[0, 1], y_axis[0, 0]-origin[0, 0], y_axis[0, 1]-origin[0, 1],
                  angles='xy', scale_units='xy', scale=1, color=(0, 1, 0, 0.3), width=0.02)  # y-axis in green

        # Apply transformation matrix to coordinate system
        transformed_origin = np.dot(origin, transformation_matrix.T)
        transformed_x_axis = np.dot(x_axis, transformation_matrix.T)
        transformed_y_axis = np.dot(y_axis, transformation_matrix.T)

        # Plot transformed coordinate system using quiver
        ax.quiver(transformed_origin[0, 0], transformed_origin[0, 1], 
                  transformed_x_axis[0, 0]-transformed_origin[0, 0], transformed_x_axis[0, 1]-transformed_origin[0, 1], 
                  angles='xy', scale_units='xy', scale=1, color='red', width=0.02)  # transformed x-axis in blue

        ax.quiver(transformed_origin[0, 0], transformed_origin[0, 1], 
                  transformed_y_axis[0, 0]-transformed_origin[0, 0], transformed_y_axis[0, 1]-transformed_origin[0, 1], 
                  angles='xy', scale_units='xy', scale=1, color='green', width=0.02)  # transformed y-axis in green



    #ax_matrix = axs[1]
    plot_matrix(ax_matrix, transformation_matrix, color_array)  # passing color_array argument
    
    
        # Add LaTeX-formatted rotation matrix above the plotted matrix
    latex_matrix = r'$T = \left[\begin{array}{ccc}' + \
    r'\cos(\theta) & -\sin(\theta) & t_x \\' + \
    r'\sin(\theta) & \cos(\theta) & t_y \\' + \
    r'0 & 0 & 1' + \
    r'\end{array}\right]$'
    ax_matrix.text(0.4, 1.6, latex_matrix, transform=ax_matrix.transAxes, fontsize=20, va='center', ha='center')
 

    plt.subplots_adjust(left=0.15, right=0.85, top=0.0, bottom=-0.5)
    plt.tight_layout()
    plt.show()

# Function to reset widget values
def reset_values(button):
    """Reset all widget values to their initial state."""
    global last_moved_slider
    scale_slider.value = 1.0
    rotation_slider.value = 0.0
    trans_x_slider.value = 0.0
    trans_y_slider.value = 0.0
    shape_select.value = 'Triangle'
    last_moved_slider = None

# Custom layout for the sliders
slider_layout = Layout(width='315px')

# Create sliders for interactive visualization with specified layout
scale_slider = FloatSlider(min=0.1, max=2.0, value=1.0, description='Scale', layout=slider_layout)
rotation_slider = FloatSlider(min=-180, max=180, value=0, description='Angle', layout=slider_layout)
trans_x_slider = FloatSlider(min=-2, max=2, value=0, description='Translation X', layout=slider_layout)
trans_y_slider = FloatSlider(min=-2, max=2, value=0, description='Translation Y', layout=slider_layout)

last_moved_slider = None  # Global variable to store the name of the last moved slider

def on_slider_change(change):
    global last_moved_slider
    # Update the global variable with the name of the moved slider
    last_moved_slider = change['owner'].description

# Attach the on_slider_change function to each slider
scale_slider.observe(on_slider_change, names='value')
rotation_slider.observe(on_slider_change, names='value')
trans_x_slider.observe(on_slider_change, names='value')
trans_y_slider.observe(on_slider_change, names='value')

# Create selection widget for shape type
shape_select = Dropdown(
    options=['Triangle', 'Coordinate System'],
    value='Triangle',
    description='Shape',
    disabled=False,
    layout=Layout(width='auto') 
)

# Create a reset button
reset_button = Button(description="Reset", layout=slider_layout)
reset_button.on_click(reset_values)

# Create a flexible space filler
filler = Label(value="", layout=Layout(flex='1 1 auto', width='auto'))
reset_box = HBox([filler, reset_button], layout=Layout(display='flex',width='auto'))

# Create a space at the top of the VBox
top_space = Label(value="", layout=Layout(height="50px"))  # Adjust height as per requirement

# Create the interactive output object
interactive_plot = interactive_output(update, {
    'shape_type': shape_select, 
    'scale': scale_slider, 
    'rotation_angle': rotation_slider,
    'trans_x': trans_x_slider, 
    'trans_y': trans_y_slider
})


# Create a container for widgets and display everything
widget_layout = Layout(display='flex', flex_flow='column', align_items='stretch', width='530px')
widget_box = VBox([
    top_space, shape_select, scale_slider, rotation_slider, 
    trans_x_slider, trans_y_slider, reset_box
], layout=widget_layout)

# Display everything
display(HBox([widget_box, interactive_plot], layout=Layout(width='100%')))  # Adjusted layout to take the full width

